In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## decay constants:

In [2]:
lambda230 = 0.0000091577*1.0014
lambda232 = 0.000000000049475
lambda234 = 0.0000028263*0.9985
lambda238 = 0.000000000155125

some abbreviations for the variable nomenclature in this script:

    conc ... concentration
    n ... number of X (typically rows in measurement)
    _e ... absolute error
    _re ... relative error
    cr ... counting rate
    ce ... counting error
    
    wt ... weighted average
    csum ... sum of counts of a given isotope
    spk ... spike
    amt ... amount of X in sample
    mr ... molar ratio /atomic ratio
    ar ... activity ratio

## sample derived input variables for the age calculation

sample, spike and chemistry blanks info

In [3]:
class Spike:
    
    def __init__(self, weight):
        
        self.weight = weight # [g]
        self.weight_e = 0.00001
        
        self.U233 = 0.78764 # [pmol/g]
        self.U233_e = 0.0002

        self.Th229 = 0.21686 # [pmol/g]
        self.Th229_e = 0.0001
        
        # spike ratios U
        self.U238_U233 = 0.016802154
        self.U235_U233 = 0.105321392
        self.U234_U233 = 0.003195127
        
        # spike ratios Th
        self.Th230_Th229 = 0.0000625
        
        self.Th232_Th229 = 0.00065
        self.Th232_Th229_e = 0.00005

In [4]:
class Sample:
    
    '''
    the init method holds default values for the chemistry blank
    at some point it might be actually useful to use the actual chemistry blank values here
    '''
    
    def __init__(self,
                 sample_weight,
                 ch_blank_238U = 0.02,
                 ch_blank_238U_e = 0.02,
                 ch_blank_232Th = 0.0005,
                 ch_blank_232Th_e = 0.0005,
                 ch_blank_230Th = 0.0001,
                 ch_blank_230Th_e = 0.0005):
        
        self.sample_weight = sample_weight # [g]
        self.sample_weight_e = 0.00001

        # chemistry blank values
        self.ch_blank_238U = ch_blank_238U # [pmol]
        self.ch_blank_238U_e = ch_blank_238U_e

        self.ch_blank_232Th = ch_blank_232Th # [pmol]
        self.ch_blank_232Th_e = ch_blank_232Th_e

        self.ch_blank_230Th = ch_blank_230Th # [fmol]
        self.ch_blank_230Th_e = ch_blank_230Th_e

        self.Th_loaded = 0.8

        self.SEM_corr = 1 # exponential for SEM correction
        self.Th_corr = 1 # frac. correction for Th, ‰ per AMU

## U measurements

In [5]:
def counting_error(counts):
    ce = 2000 / counts ** 0.5
    return ce

class U_measurement:
    
    def __init__(self,
                 mean_235U_233U,
                 mean_235U_233U_e,
                 mean_234U_235U,
                 mean_234U_235U_e,
                 mean_233U,
                 n_233,
                 n_234U_235U,
                 bg_233U,
                 bg_234U,
                 bg_235U):
        
        # ratio means ... this will later be the output of the standard deviation filters
        # this is the green fields in Hai's template
        
        self.mean_235U_233U = mean_235U_233U
        self.mean_235U_233U_e = mean_235U_233U_e
        self.mean_235U_233U_re = self.mean_235U_233U_e / self.mean_235U_233U * 1000

        self.mean_234U_235U = mean_234U_235U
        self.mean_234U_235U_e = mean_234U_235U_e
        
        self.mean_233U = mean_233U
        self.mean_233U_e = self.mean_233U * 0.05
        
        self.n_233 = n_233
        self.n_234U_235U = n_234U_235U
        
        # background counts [cps]
        # this is the yellow fields

        self.bg_233U = bg_233U
        self.bg_234U = bg_234U
        self.bg_235U = bg_235U

        # some static variables
        
        self.fil_blk_238U = 0.0001
        self.fil_blk_238U_e = 0.1
        
        self.r_238U_235U = 137.82
        
        # counting times + errors
        
        self.ct_time_233U = 0.131 * 3
        self.ct_time_235U = 0.131
        self.ct_time_234U = 1.049
        
        self.csum_234 = 3888322 # this requires U ratios to be calculated first as an input - DO LATER
        
        self.ce233 = counting_error(self.mean_233U * self.n_233 * self.ct_time_233U)
        self.ce234 = counting_error(self.csum_234)
        self.ce235 = counting_error(self.mean_233U * self.n_233 * self.ct_time_235U * self.mean_235U_233U)
        
        ratios = self.mean_235U_233U * self.mean_234U_235U * self.mean_233U
        self.wt_234U_235U = self.mean_234U_235U * (1 - self.bg_234U / ratios)
        self.wt_234U_235U_e = self.mean_234U_235U_e
        
        
    def r_234U_233U(self):
        r = (self.wt_234U_235U * self.mean_235U_233U) ** 1 # exponent is SEM_corr from Sample class
        return r
    
    def r_234U_235U(self):
        
        mean_2335_234 = 0.00000000001
        r_tailcorr = self.wt_234U_235U * (1-(4/9*mean_2335_234 + 5/9*mean_2335_234))        

## Th measurements

In [6]:
class Th_measurement:
    
    def __init__(self,
                 mean_230Th_229Th,
                 mean_230Th_229Th_e, 
                 mean_232Th_229Th,
                 mean_232Th_229Th_e, 
                 mean_229Th,
                 n_229,
                 bg_230Th):
        
        #ratio means ... as above, take these from filter function
        self.mean_230Th_229Th = mean_230Th_229Th
        self.mean_230Th_229Th_e = mean_230Th_229Th_e
        self.mean_230Th_229Th_re = self.mean_230Th_229Th_e / self.mean_230Th_229Th * 1000

        self.mean_232Th_229Th = mean_232Th_229Th
        self.mean_232Th_229Th_e = mean_232Th_229Th_e
        self.mean_232Th_229Th_re = mean_232Th_229Th_e / mean_232Th_229Th * 1000

        self.mean_229Th = mean_229Th
        self.n_229 = n_229
        
        self.bg_230Th = bg_230Th # aka darknoise(cpm); attention! cps * 60 for cpm from wash file
        self.bg_230Th_re = 200
        self.bg_230Th_e = self.bg_230Th * self.bg_230Th_re / 1000
        
        self.run_duration = 60 # [min]
        
        ############################################################################################
        # abundance sensitivity
        # @1 amu
        self.Th_AS1 = 0.0000000001
        self.Th_AS1_re = 250
        self.Th_AS1_e = self.Th_AS1_re / 1000 * self.Th_AS1
        # @2 amu
        self.Th_AS2 = self.Th_AS1 / 2.5
        self.Th_AS2_re = 250
        self.Th_AS2_e = self.Th_AS2_re / 1000 * self.Th_AS2

        ############################################################################################
        # counting times

        self.cr_229Th = self.mean_229Th
        self.cr_229Th_e = 10

        self.cr_230Th = self.cr_229Th * self.mean_230Th_229Th
        self.cr_230Th_e = 10

        self.cr_232Th = self.cr_229Th * self.mean_232Th_229Th
        self.cr_232Th_e = 10

        self.mv_232Th = self.cr_229Th * self.mean_232Th_229Th / 62500

## initialize with dummy data

In [7]:
Th_input = {'mean_230Th_229Th' : 0.035835,
            'mean_230Th_229Th_e' : 0.0001,
            'mean_232Th_229Th' : 0.0315347814243728,
            'mean_232Th_229Th_e' : 0.00530355622869084,
            'mean_229Th' : 350022,
            'n_229' : 16,
            'bg_230Th' : 60}

U_input = {'mean_235U_233U' : 4.298,
           'mean_235U_233U_e' : 0.00317,
           'mean_234U_235U' : 0.008368,
           'mean_234U_235U_e' : 0.000007,
           'mean_233U' : 103277,
           'n_233' : 998,
           'n_234U_235U' : 869,
           'bg_233U' : 10,
           'bg_234U' : 0.1,
           'bg_235U' : 160}

In [8]:
Spk = Spike(0.5)
smpl = Sample(0.0521)
U = U_measurement(**U_input)
Th = Th_measurement(**Th_input)

## U/Th calculations

In [10]:
# ratios from measurements

def activity_ratio(mr, dc1, dc2):
    return mr * dc1 / dc2

def molar_ratio(ar, dc1, dc2):
    return ar * dc2 / dc1

mr_232Th_238U = 0.0000125
mr_230Th_238U = 0.0000170
mr_230Th_234U = 0.3004076

ar_232Th_238U = activity_ratio(mr_232Th_238U, lambda232, lambda238)
ar_230Th_238U = activity_ratio(mr_230Th_238U, lambda230, lambda238)
ar_230Th_234U = activity_ratio(mr_230Th_234U, lambda230, lambda234)

In [13]:
ar_230Th_234U = 0.387168317640528
ar_234U_238U = 2.12389700300915
ar_232Th_238Th = 0.0359724159166239
ar_230Th_232Th = 22.8593384276089

molar_ratio(ar_230Th_232Th,lambda232, lambda230)

4237130.63299525